# Forecast Harga Pasar Komoditas Bawang Merah di Brebes

## Import Libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Flatten

## Data Collections

In [2]:
file_path = "dataset.csv"
df = pd.read_csv(file_path)
df

Komoditas (Rp)  15/03/2021 16/03/2021 17/03/2021  \
0                       Beras Premium     11.500     11.500     11.500   
1                        Beras Medium      9.000      9.000      9.000   
2         Kedelai Biji Kering (Impor)     11.000     11.000     11.000   
3                        Bawang Merah     28.000     28.000     28.000   
4                Bawang Putih Bonggol     22.000     22.000     22.000   
5                Cabai Merah Keriting     50.000     50.000     50.000   
6                   Cabai Rawit Merah    115.000    115.000    115.000   
7                   Daging Sapi Murni    120.000    120.000    120.000   
8                     Daging Ayam Ras     36.000     36.000     36.000   
9                      Telur Ayam Ras     22.000     22.000     22.000   
10                      Gula Konsumsi     12.500     12.500     12.500   
11    Minyak Goreng Kemasan Sederhana     14.000     14.000     14.000   
12              Tepung Terigu (Curah)      7.000      7.000      7.000   
13                Minyak Goreng Curah          -          -          -   
14                 Jagung Tk Peternak          -          -          -   
15                       Ikan Kembung          -          -          -   
16                       Ikan Tongkol          -          -          -   
17                       Ikan Bandeng          -          -          -   
18              Garam Halus Beryodium          -          -          -   
19  Tepung Terigu Kemasan (non-curah)          -          -          -   
20                         Beras SPHP          -          -          -   

   18/03/2021 19/03/2021 20/03/2021 21/03/2021 22/03/2021 23/03/2021  ...  \
0      11.500     11.500     11.500          -     11.500          -  ...   
1       9.000      9.000      9.000          -      9.000          -  ...   
2      11.000     11.000     11.000          -     10.000          -  ...   
3      28.000     28.000     27.000          -     27.000          -  ...   
4      22.000     22.000     22.000          -     22.000          -  ...   
5      50.000     50.000     50.000          -     50.000          -  ...   
6     115.000    110.000    115.000          -    115.000          -  ...   
7     120.000    120.000    120.000          -    120.000          -  ...   
8      36.000     36.000     36.000          -     36.000          -  ...   
9      22.000     22.000     22.000          -     22.000          -  ...   
10     12.500     12.500     12.500          -     12.500          -  ...   
11     14.000     14.000     14.000          -     14.000          -  ...   
12      7.000      7.000      7.000          -      7.000          -  ...   
13          -          -          -          -          -          -  ...   
14          -          -          -          -          -          -  ...   
15          -          -          -          -          -          -  ...   
16          -          -          -          -          -          -  ...   
17          -          -          -          -          -          -  ...   
18          -          -          -          -          -          -  ...   
19          -          -          -          -          -          -  ...   
20          -          -          -          -          -          -  ...   

   30/11/2024 01/12/2024 02/12/2024 03/12/2024 04/12/2024 05/12/2024  \
0        15.0       15.0       15.0       15.0       15.0       15.0   
1        13.5       13.5       13.5       13.5       13.5       13.5   
2        10.3       10.3       10.3       10.3       10.3       10.3   
3        35.0       35.0       35.0       33.0       33.0       32.0   
4        35.0       35.0       35.0       35.0       35.0       35.0   
5        18.0       18.0       22.0       22.0       22.0       24.0   
6        32.0       32.0       35.0       35.0       35.0       35.0   
7       143.0      143.0      143.0      143.0      143.0      143.0   
8        39.0       39.0       39.0       39.0    

## Data Cleaning

In [3]:
# Menambah label "tanggal" ke dalam list kolom
kolom = ["tanggal"] + [x.strip() for x in df[" Komoditas (Rp) "]]

# Ubah nama kolom pada df_transformed sesuai dengan kolom yang diinginkan
df_transformed = df.set_index(" Komoditas (Rp) ").T.reset_index()

df_transformed.columns = kolom
# mengubah tanggal menjadi tipe data tanggal
df_transformed['tanggal'] = pd.to_datetime(df_transformed['tanggal'], format='%d/%m/%Y')

# Mengganti nilai kosong atau tanda tertentu dengan NaN
df_transformed.replace(["", "-", " "], np.nan, inplace=True)

# Mengonversi kolom data (selain tanggal) menjadi float
for col in df_transformed.columns[1:]:  # Mulai dari indeks 1 agar kolom 'tanggal' tidak diubah
    df_transformed[col] = pd.to_numeric(df_transformed[col], errors='coerce') * 1000

# Memeriksa apakah perubahan berhasil
print(df_transformed.dtypes)
print(df_transformed.isnull().sum())
df_transformed

tanggal                              datetime64[ns]
Beras Premium                               float64
Beras Medium                                float64
Kedelai Biji Kering (Impor)                 float64
Bawang Merah                                float64
Bawang Putih Bonggol                        float64
Cabai Merah Keriting                        float64
Cabai Rawit Merah                           float64
Daging Sapi Murni                           float64
Daging Ayam Ras                             float64
Telur Ayam Ras                              float64
Gula Konsumsi                               float64
Minyak Goreng Kemasan Sederhana             float64
Tepung Terigu (Curah)                       float64
Minyak Goreng Curah                         float64
Jagung Tk Peternak                          float64
Ikan Kembung                                float64
Ikan Tongkol                                float64
Ikan Bandeng                                float64
Garam Halus 

C:\Users\ACER\AppData\Local\Temp\ipykernel_22568\1358906317.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_transformed.replace(["", "-", " "], np.nan, inplace=True)


tanggal  Beras Premium  Beras Medium  Kedelai Biji Kering (Impor)  \
0    2021-03-15        11500.0        9000.0                      11000.0   
1    2021-03-16        11500.0        9000.0                      11000.0   
2    2021-03-17        11500.0        9000.0                      11000.0   
3    2021-03-18        11500.0        9000.0                      11000.0   
4    2021-03-19        11500.0        9000.0                      11000.0   
...         ...            ...           ...                          ...   
1361 2024-12-05        15000.0       13500.0                      10300.0   
1362 2024-12-06        15000.0       13500.0                      10300.0   
1363 2024-12-07        15000.0       13500.0                      10300.0   
1364 2024-12-08        15000.0       13500.0                      10300.0   
1365 2024-12-09        15000.0       13500.0                      10300.0   

      Bawang Merah  Bawang Putih Bonggol  Cabai Merah Keriting  \
0          28000.0               22000.0               50000.0   
1          28000.0               22000.0               50000.0   
2          28000.0               22000.0               50000.0   
3          28000.0               22000.0               50000.0   
4          28000.0               22000.0               50000.0   
...            ...                   ...                   ...   
1361       32000.0               35000.0               24000.0   
1362       32000.0               35000.0               24000.0   
1363       32000.0               35000.0               24000.0   
1364       32000.0               35000.0               24000.0   
1365       32000.0               35000.0               32000.0   

      Cabai Rawit Merah  Daging Sapi Murni  Daging Ayam Ras  ...  \
0              115000.0           120000.0          36000.0  ...   
1              115000.0           120000.0          36000.0  ...   
2              115000.0           120000.0          36000.0  ...   
3              115000.0           120000.0          36000.0  ...   
4              110000.0           120000.0          36000.0  ...   
...                 ...                ...              ...  ...   
1361            35000.0           143000.0          40000.0  ...   
1362            35000.0           143000.0          40000.0  ...   
1363            35000.0           143000.0          40000.0  ...   
1364            35000.0           143000.0          40000.0  ...   
1365            40000.0           143000.0          40000.0  ...   

      Minyak Goreng Kemasan Sederhana  Tepung Terigu (Curah)  \
0                             14000.0                 7000.0   
1                             14000.0                 7000.0   
2                             14000.0                 7000.0   
3                             14000.0                 7000.0   
4                             14000.0                 7000.0   
...                               ...                    ...   
1361                          18000.0                 9000.0   
1362                          18000.0                 9000.0   
1363                          18000.0                 9000.0   
1364                          18000.0                 9000.0   
1365                          18000.0                 9000.0   

      Minyak Goreng Curah  Jagung Tk Peternak  Ikan Kembung  Ikan Tongkol  \
0                     NaN                 NaN           NaN           NaN   
1                     NaN                 NaN           NaN           NaN   
2                     NaN                 NaN           NaN           NaN   
3                     NaN                 NaN           NaN           NaN   
4                     NaN                 NaN           NaN           NaN   
...                   ...                 ...           ...           ...   
1361              17000.0              5100.0       30000.0       30000.0   
1362              17000.0              5100.0       30000.0       30000.0   
1363       

In [4]:
# Buat dataframe barang
df_barang = pd.DataFrame()
df_barang['tanggal'] = df_transformed['tanggal']
df_barang['harga'] = df_transformed['Bawang Merah']

# Isi null pada data
df_barang['harga'] = df_barang['harga'].interpolate(method='linear').ffill()

# Buat kolom minggu dari tanggal
df_barang['3 Hari'] = (df_barang.index // 3) + 1

#kelompokkan berdasarkan minggu
weekly_data = df_barang.groupby(['3 Hari']).agg({
    'harga': 'mean'  # Rata-rata harga per minggu
}).reset_index()
weekly_data.rename(columns={'harga': 'Harga/3 hari'}, inplace=True)

weekly_data

3 Hari  Harga/3 hari
0         1  28000.000000
1         2  27666.666667
2         3  27000.000000
3         4  29000.000000
4         5  30000.000000
..      ...           ...
451     452  35000.000000
452     453  35000.000000
453     454  32666.666667
454     455  32000.000000
455     456  32000.000000

[456 rows x 2 columns]

In [5]:
plt.plot(weekly_data['Harga/3 hari'])
plt.title('Harga berdasarkan Minggu')
plt.grid(True)

## Data Preparation

In [7]:
week = np.array(weekly_data["3 Hari"])
series = np.array(weekly_data['Harga/3 hari'])
week.shape, series.shape

((456,), (456,))

In [8]:
model.save('forecast_3hari_1.keras')

NameError: name 'model' is not defined

### Future Forecast

In [17]:
window_size = 24

In [18]:
def scaled_price(data):
    return data/1000

In [19]:
def inverse_scale(data):
    return data*1000

1. Load model

In [20]:
loaded_model = tf.keras.models.load_model('model_prediksi_3hari.keras')

In [21]:
print(loaded_model.input_shape)

(None, 24, 1)


2. tes pakai data 30 index yang lalu

In [22]:
actual = series[:-30]
seq_res = np.array([])

# seq_res = predict_n_weeks(4, actual, seq_res)
# prediction(actual)
# seq_res

In [23]:
series.shape

(456,)

In [24]:
act_temp = actual.reshape(-1, 1)

In [25]:
# data dibagi 1000
actual_scaled = scaled_price(act_temp)
# ambil 24 data terakhir
seq = actual_scaled[-window_size:]
# prediksi
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
# hasil prediksi dikali 1000
hasil = inverse_scale(hasil)
# hasil prediksi 3 hari pertama masuk ke variabell seq_res
seq_res = np.append(seq_res, hasil.item())
# hasil predksi masuk ke variabel pertama yang mengambil 24 data terakhir
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


In [26]:
seq_res

array([19909.6171875])

In [27]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


3. Contoh hasil prediksi 6 hari(3 hari pertama dan 3 hari kedua)

In [28]:
seq_res

array([19909.6171875 , 20113.04882812])

4. harga asli selama 6 hari (3 hari pertama dan 3 hari kedua)

In [29]:
print(weekly_data[(weekly_data['3 Hari'] > 412) & (weekly_data['3 Hari'] <= 414)])

     3 Hari  Harga/3 hari
412     413       18000.0
413     414       18000.0


5. input data asli

In [30]:
actual = np.append(actual, [18000.00, 18000.00])

6. looping lagi

In [31]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [32]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [33]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625])

In [34]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [35]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938])

In [36]:
print(weekly_data[(weekly_data['3 Hari'] > 414) & (weekly_data['3 Hari'] <= 416)])

     3 Hari  Harga/3 hari
414     415       18000.0
415     416       18000.0


In [37]:
actual = np.append(actual, [18000.00, 18000.00])

In [38]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [39]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [40]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188])

In [41]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [42]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  ])

In [43]:
print(weekly_data[(weekly_data['3 Hari'] > 416) & (weekly_data['3 Hari'] <= 418)])

     3 Hari  Harga/3 hari
416     417  17333.333333
417     418  17000.000000


In [44]:
actual = np.append(actual, [17333.33, 17000.00])

In [45]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [46]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [47]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  ])

In [48]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [49]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ])

In [50]:
print(weekly_data[(weekly_data['3 Hari'] > 418) & (weekly_data['3 Hari'] <= 420)])

     3 Hari  Harga/3 hari
418     419       18000.0
419     420       20000.0


In [51]:
actual = np.append(actual, [18000.00, 20000.00])

In [52]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [53]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [54]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   ])

In [55]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [56]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625])

In [57]:
print(weekly_data[(weekly_data['3 Hari'] > 420) & (weekly_data['3 Hari'] <= 422)])

     3 Hari  Harga/3 hari
420     421       20000.0
421     422       20000.0


In [58]:
actual = np.append(actual, [20000.00, 20000.14])

In [59]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [60]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [61]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     ])

In [62]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [63]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ])

In [64]:
print(weekly_data[(weekly_data['3 Hari'] > 422) & (weekly_data['3 Hari'] <= 424)])

     3 Hari  Harga/3 hari
422     423  20000.000000
423     424  23333.333333


In [65]:
actual = np.append(actual, [20000.00, 23333.00])

In [66]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [67]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [68]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625])

In [69]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [70]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688])

In [71]:
print(weekly_data[(weekly_data['3 Hari'] > 424) & (weekly_data['3 Hari'] <= 426)])

     3 Hari  Harga/3 hari
424     425       20000.0
425     426       20000.0


In [72]:
actual = np.append(actual, [20000.00, 20000.00])

In [73]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [74]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [75]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188])

In [76]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [77]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ])

In [78]:
print(weekly_data[(weekly_data['3 Hari'] > 426) & (weekly_data['3 Hari'] <= 428)])

     3 Hari  Harga/3 hari
426     427       18000.0
427     428       18500.0


In [79]:
actual = np.append(actual, [18000.00, 18500.00])

In [80]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [81]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [82]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 ])

In [83]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [84]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 , 18673.125     ])

In [85]:
print(weekly_data[(weekly_data['3 Hari'] > 428) & (weekly_data['3 Hari'] <= 430)])

     3 Hari  Harga/3 hari
428     429       20000.0
429     430       20000.0


In [86]:
actual = np.append(actual, [20000.00, 20000.00])

In [87]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [88]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [89]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 , 18673.125     , 20096.2109375 ])

In [90]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


In [91]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 , 18673.125     , 20096.2109375 , 20263.09765625])

In [92]:
print(weekly_data[(weekly_data['3 Hari'] > 430) & (weekly_data['3 Hari'] <= 432)])

     3 Hari  Harga/3 hari
430     431       20000.0
431     432       20000.0


In [93]:
actual = np.append(actual, [20000.00, 20000.00])

In [94]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [95]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [96]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 , 18673.125     , 20096.2109375 , 20263.09765625,
       20994.34960938])

In [97]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [98]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 , 18673.125     , 20096.2109375 , 20263.09765625,
       20994.34960938, 20980.8046875 ])

In [99]:
print(weekly_data[(weekly_data['3 Hari'] > 432) & (weekly_data['3 Hari'] <= 434)])

     3 Hari  Harga/3 hari
432     433       20000.0
433     434       20000.0


In [100]:
actual = np.append(actual, [20000.00, 20000.00])

In [101]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [102]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [103]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 , 18673.125     , 20096.2109375 , 20263.09765625,
       20994.34960938, 20980.8046875 , 20326.19921875])

In [104]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [105]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 , 18673.125     , 20096.2109375 , 20263.09765625,
       20994.34960938, 20980.8046875 , 20326.19921875, 20481.67578125])

In [106]:
print(weekly_data[(weekly_data['3 Hari'] > 434) & (weekly_data['3 Hari'] <= 436)])

     3 Hari  Harga/3 hari
434     435  20000.000000
435     436  21333.333333


In [107]:
actual = np.append(actual, [20000.00, 21333.33])

In [108]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [109]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [110]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [111]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 , 18673.125     , 20096.2109375 , 20263.09765625,
       20994.34960938, 20980.8046875 , 20326.19921875, 20481.67578125,
       21755.98046875, 22231.39257812])

In [112]:
print(weekly_data[(weekly_data['3 Hari'] > 436) & (weekly_data['3 Hari'] <= 438)])

     3 Hari  Harga/3 hari
436     437       22000.0
437     438       22000.0


In [113]:
actual = np.append(actual, [22000.00, 22000.00])

In [114]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [115]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [116]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [117]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 , 18673.125     , 20096.2109375 , 20263.09765625,
       20994.34960938, 20980.8046875 , 20326.19921875, 20481.67578125,
       21755.98046875, 22231.39257812, 22145.68945312, 22086.52734375])

In [118]:
print(weekly_data[(weekly_data['3 Hari'] > 438) & (weekly_data['3 Hari'] <= 440)])

     3 Hari  Harga/3 hari
438     439       22000.0
439     440       22000.0


In [119]:
actual = np.append(actual, [22000.00, 22000.00])

In [120]:
act_temp = actual.reshape(-1, 1)
# seq_res = np.array([])

In [121]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [122]:
actual_scaled = scaled_price(act_temp)
seq = actual_scaled[-window_size:]
hasil = loaded_model.predict(seq.reshape(1, window_size, 1))
hasil = inverse_scale(hasil)
seq_res = np.append(seq_res, hasil.item())
act_temp = np.append(act_temp, np.expand_dims([hasil.item()], axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [123]:
seq_res

array([19909.6171875 , 20113.04882812, 18254.56640625, 18657.28710938,
       18275.51367188, 18405.234375  , 16988.453125  , 17277.4765625 ,
       20326.78125   , 20595.91015625, 20529.375     , 21022.421875  ,
       23569.87890625, 23815.62304688, 19766.70117188, 20078.78125   ,
       18670.0859375 , 18673.125     , 20096.2109375 , 20263.09765625,
       20994.34960938, 20980.8046875 , 20326.19921875, 20481.67578125,
       21755.98046875, 22231.39257812, 22145.68945312, 22086.52734375,
       21949.90625   , 22245.44335938])

In [124]:
seq_res.shape

(30,)

In [125]:
plt.plot(seq_res, label='prediction')
plt.plot(actual[-30:], label='actual')
plt.legend()

In [126]:
mae = mean_absolute_error(seq_res, actual[-30:])
mse = mean_squared_error(seq_res, actual[-30:])
rmse = np.sqrt(mse)

print(f'mae: {mae:.2f}')
print(f'mse: {mse:.2f}')
print(f'rmse: {rmse:.2f}')

mae: 614.16
mse: 930459.59
rmse: 964.60


In [127]:
mape = np.mean(np.abs((seq_res - actual[-30:]) / seq_res)) * 100
accuracy = 100 - mape
print(f"Akurasi prediksi: {accuracy:.2f}%")

Akurasi prediksi: 97.07%
